In [63]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from functools import reduce
from api_keys import geoapify_key
import json
import requests

In [137]:
dataset_2016 = 'Burlington_StatsCanada_2016_Census.csv'
dataset_2021 = 'Burlington_StatsCanada_2021_Census.csv'
private_schools = 'Private School Boards Ontario.csv'
public_schools = 'Public School Boards Ontario.csv'
house_sale_median = 'Burlington_Median_Sale_Price.csv'
house_rent_median = 'Burlington_Median_Rental_Price.csv'
house_sale_average = 'Burlington_Avg_Sale_Price.csv'
house_rent_average = 'Burlington_Avg_Rental_Price.csv'
df_2016 = pd.read_csv(dataset_2016, encoding = "ISO-8859-1", on_bad_lines = 'skip', low_memory = True, engine = 'python')
df_2021 = pd.read_csv(dataset_2021, encoding = "ISO-8859-1", on_bad_lines = 'skip', low_memory = True, engine = 'python')
df_private_schools = pd.read_csv(private_schools, encoding = "ISO-8859-1", on_bad_lines = 'skip', low_memory = True, engine = 'python')
df_public_schools = pd.read_csv(public_schools, encoding = "ISO-8859-1", on_bad_lines = 'skip', low_memory = True, engine = 'python')
df_sale_median = pd.read_csv(house_sale_median, encoding = "ISO-8859-1", on_bad_lines = 'skip', low_memory = True, engine = 'python')
df_rent_median = pd.read_csv(house_rent_median, encoding = "ISO-8859-1", on_bad_lines = 'skip', low_memory = True, engine = 'python')
df_sale_average = pd.read_csv(house_sale_average, encoding = "ISO-8859-1", on_bad_lines = 'skip', low_memory = True, engine = 'python')
df_rent_average = pd.read_csv(house_rent_average, encoding = "ISO-8859-1", on_bad_lines = 'skip', low_memory = True, engine = 'python')
df_public_schools.sample(3)

,ï»¿Board Number,Board Name,Board Type,School Number,School Name,School Type,School Special Condition Code,School Level,School Language,Grade Range,...,Change in Grade 6 Writing Achievement Over Three Years,Percentage of Grade 6 Students Achieving the Provincial Standard in Mathematics,Change in Grade 6 Mathematics Achievement Over Three Years,Percentage of Grade 9 Students Achieving the Provincial Standard in Mathematics,Change in Grade 9 Mathematics Achievement Over Three Years,Percentage of Students That Passed the Grade 10 OSSLT on Their First Attempt,Change in Grade 10 OSSLT Literacy Achievement Over Three Years,Percentage of School-Aged Children Who Live in Low-Income Households,"Percentage of Students Whose Parents Have No Degree, Diploma or Certificate",Extract Date
3915,B66052,Toronto DSB,Pub Dist Sch Brd (E/F),336580,Mountview Alternative School School,Public,Not applicable,Elementary,English,JK-8,...,NaN,71%,NaN,NaN,NaN,NaN,NaN,10,0,3-Nov-23
3856,B66052,Toronto DSB,Pub Dist Sch Brd (E/F),288543,Keelesdale Junior Public School,Public,Not applicable,Elementary,English,JK-6,...,NaN,25%,NaN,NaN,NaN,NaN,NaN,30,15,3-Nov-23
3247,B66044,Thames Valley DSB,Pub Dist Sch Brd (E/F),931934,Oakridge Secondary School,Public,Not applicable,Secondary,English,9-12,...,NaN,NaN,NaN,70%,NaN,89%,NaN,16,3,3-Nov-23


In [28]:
housing_dfs = [df_sale_median, df_rent_median, df_sale_average, df_rent_average]
df_housing_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Primary Year'], how='outer'), housing_dfs)
df_housing_merged.head()

,Primary Year,"Sale Price, Median","Original Price, Median","Sale Price, Average","Original Price, Average"
0,2016,"$609,000","$2,400","$663,675","$2,710"
1,2017,"$690,000","$2,400","$763,549","$2,779"
2,2018,"$660,000","$2,400","$735,275","$2,711"
3,2019,"$690,000","$2,500","$755,017","$2,636"
4,2020,"$800,000","$2,500","$877,311","$2,683"


In [56]:
private_school_count = len(df_private_schools.loc[df_private_schools['City'] == 'Burlington'])
print(private_school_count)

2


In [59]:
public_school_count = len(df_public_schools.loc[(df_public_schools['City'] == 'Burlington') & (df_public_schools['School Type'] == 'Public')])
catholic_school_count = len(df_public_schools.loc[(df_public_schools['City'] == 'Burlington') & (df_public_schools['School Type'] == 'Catholic')])
print(public_school_count)
print(catholic_school_count)

38
17


In [70]:
target_city = 'Burlington, ON, Canada'

qp = {
    "text": target_city,
    "apiKey": geoapify_key
}
base_url = "https://api.geoapify.com/v2/places"
geocode_url = 'https://api.geoapify.com/v1/geocode/search'
target_url = f"https://api.geoapify.com/v1/geocode/search?text={target_city}&format=json&apiKey={geoapify_key}"
response = requests.get(geocode_url, params=qp).json()
from pprint import pprint
pprint(response)



{'features': [{'bbox': [-79.9592676, 43.2862845, -79.713939, 43.4765533],
               'geometry': {'coordinates': [-79.7966835, 43.3248924],
                            'type': 'Point'},
               'properties': {'address_line1': 'Burlington, ON',
                              'address_line2': 'Canada',
                              'category': 'administrative',
                              'city': 'Burlington',
                              'country': 'Canada',
                              'country_code': 'ca',
                              'county': 'Halton Region',
                              'datasource': {'attribution': '© OpenStreetMap '
                                                            'contributors',
                                             'license': 'Open Database License',
                                             'sourcename': 'openstreetmap',
                                             'url': 'https://www.openstreetmap.org/copyright'},
        

In [71]:
lat = response['features'][0]['properties']['lat']
lon = response['features'][0]['properties']['lon']

print(lat, lon)

43.3248924 -79.7966835


{'features': [{'geometry': {'coordinates': [-79.79813476586828,
                                            43.32161884859703],
                            'type': 'Point'},
               'properties': {'address_line1': 'Spencer Smith Park',
                              'address_line2': 'Burlington, ON L7S 1B1, Canada',
                              'categories': ['leisure', 'leisure.park'],
                              'city': 'Burlington',
                              'country': 'Canada',
                              'country_code': 'ca',
                              'county': 'Halton Region',
                              'datasource': {'attribution': '© OpenStreetMap '
                                                            'contributors',
                                             'license': 'Open Database Licence',
                                             'raw': {'leisure': 'park',
                                                     'name': 'Spencer Smith '
     

In [114]:
#for index, row in leisure_df.iterrows():

#print(leisure_df)
categories = 'leisure'
radius = 300000
params = {
    "categories": categories,
    "apiKey": geoapify_key
}
params["filter"] = f'circle:{lon},{lat},{radius}'
params["bias"] = f'proximity:{lon},{lat}'
leisure_dict = requests.get(base_url, params=params).json()

leisure_types = []
for feature in leisure_dict['features']:
    leisure_info = feature['properties']['datasource']['raw']['osm_id']
    leisure_types.append(leisure_info)

leisure_count = len(leisure_types)

print(leisure_count)


20


In [167]:
def format_value(value, percentage=False, currency=False):
    if isinstance(value, np.ndarray):
        value = value.item()
    if percentage:
        return f"{value:.2f}%"
    elif currency:
        return f"${value:.2f}"
    else:
        return str(round(value))
    
pop_2016 = df_2016.loc[df_2016['Characteristics'] == 'Population; 2016']['Total'].values
income_2016 = df_2016.loc[df_2016['Characteristics'] == '    Average total income in 2015 among recipients ($)']['Total'].values
house_value_2016 = df_2016.loc[df_2016['Characteristics'] == '  Average value of dwellings ($)']['Total'].values
unemployment_2016 = df_2016.loc[df_2016['Characteristics'] == 'Unemployment rate']['Total'].values
rented_2016 = df_2016.loc[df_2016['Characteristics'] == '  Renter']['Total'].values
owned_2016 = df_2016.loc[df_2016['Characteristics'] == '  Owner']['Total'].values
burlington_2016 = {
    'Year': '2016',
    'Population': format_value(pop_2016),
    'Household Income': format_value(income_2016, currency = True),
    'House Values': format_value(house_value_2016, currency = True),
    'Unemployment Rate': format_value(unemployment_2016, percentage = True),
    'Rented Housing': format_value(rented_2016),
    'Owned Housing': format_value(owned_2016)
}
burlington_2016_df = pd.DataFrame(burlington_2016, index = [0])
print(burlington_2016_df)

   Year Population Household Income House Values Unemployment Rate  \
0  2016     183314        $60898.00   $632556.00             5.60%   

  Rented Housing Owned Housing  
0          16835         54540  


In [169]:
pop_2021 = df_2021.loc[df_2021['Characteristic'] == 'Population, 2021']['Total'].values
income_2021 = df_2021.loc[df_2021['Characteristic'] == '    Average total income in 2019 among recipients ($)']['Total'].values
house_value_2021 = df_2021.loc[df_2021['Characteristic'] == '  Average value of dwellings ($)']['Total'].values
unemployment_2021 = df_2021.loc[df_2021['Characteristic'] == 'Unemployment rate']['Total'].values
rented_2021 = df_2021.loc[df_2021['Characteristic'] == '  Renter']['Total'].values
owned_2021 = df_2021.loc[df_2021['Characteristic'] == '  Owner']['Total'].values



burlington_2021 = {
    'Year': '2021',
    'Population': format_value(pop_2021),
    'Household Income': format_value(income_2021, currency = True),
    'House Values': format_value(house_value_2021, currency = True),
    'Unemployment Rate': format_value(unemployment_2021, percentage = True),
    'Rented Housing': format_value(rented_2021),
    'Owned Housing': format_value(owned_2021)
}
burlington_2021_df = pd.DataFrame(burlington_2021, index = [0])
print(burlington_2021_df)

   Year Population Household Income House Values Unemployment Rate  \
0  2021     186948        $67200.00  $1024000.00            10.70%   

  Rented Housing Owned Housing  
0          18220         54955  


In [170]:
amenities = {
    '# of Public Schools': public_school_count,
    '# of Catholic Schools': catholic_school_count,
    '# of Private Schools': private_school_count,
    '# of Leisure Spaces': leisure_count
}

amenities_df = pd.DataFrame(amenities, index = [0])
print(amenities_df)

   # of Public Schools  # of Catholic Schools  # of Private Schools  \
0                   38                     17                     2   

   # of Leisure Spaces  
0                   20  


In [171]:
combined_df = pd.concat([burlington_2016_df, burlington_2021_df], ignore_index=True)
combined_df.set_index(['Year'])
print(combined_df)

   Year Population Household Income House Values Unemployment Rate  \
0  2016     183314        $60898.00   $632556.00             5.60%   
1  2021     186948        $67200.00  $1024000.00            10.70%   

  Rented Housing Owned Housing  
0          16835         54540  
1          18220         54955  
